# Car Detection

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/car_detection.ipynb)

## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [109]:
pip install geoai-py

## Import libraries

In [110]:
import geoai

## Download sample data

We will download a sample image from Hugging Face Hub to use for car detection. You can find more high-resolution images from [OpenAerialMap](https://openaerialmap.org).

In [111]:
raster_url = r"C:\Users\zhang\Desktop\GEOAI\finalcars_7cm.tif"


In [112]:
raster_path = geoai.download_file(raster_url)

File already exists: cars_7cm.tif


## Visualize the image

In [113]:
geoai.view_raster(raster_url)

## Initialize the model

In [114]:
detector = geoai.CarDetector()

Model path not specified, downloading from Hugging Face...
Model downloaded to: /root/.cache/huggingface/hub/models--giswqs--geoai/snapshots/089548329c81f128fa12576663e7abdedb5cfa0e/car_detection_usa.pth
Model loaded successfully


## Extract cars

Extract cars from the image using the model and save the output image.

In [115]:
mask_path = detector.generate_masks(
    raster_path=raster_path,
    output_path="cars_masks.tif",
    confidence_threshold=0.3,
    mask_threshold=0.5,
    overlap=0.25,
    chip_size=(400, 400),
)

Dataset initialized with 15 rows and 28 columns of chips
Image dimensions: 8351 x 4463 pixels
Chip size: 400 x 400 pixels
Overlap: 25.0% (stride_x=300, stride_y=300)
CRS: EPSG:3857
Processing raster with 105 batches


100%|██████████| 105/105 [00:46<00:00,  2.27it/s]


Masks with confidence values saved to cars_masks.tif


Convert the image masks to polygons and save the output GeoJSON file.

In [116]:
gdf = detector.vectorize_masks(
    masks_path="cars_masks.tif",
    output_path="cars.geojson",
    min_object_area=100,
    max_object_area=2000,
)

Processing masks from: cars_masks.tif
Found 1012 connected components
Using single-threaded processing, you can speed up processing by setting n_workers > 1


Processing components: 100%|██████████| 1012/1012 [01:27<00:00, 11.55it/s]

Saved 101 objects with confidence to cars.geojson
Total processing time: 88.02 seconds


## Add geometric properties

In [117]:
gdf = geoai.add_geometric_properties(gdf)

## Visualize initial results

In [118]:
geoai.view_vector_interactive(gdf, column="confidence", tiles=raster_url)

## Filter cars by area

In [119]:
gdf_filter = gdf[
    (gdf["area_m2"] > 8) & (gdf["area_m2"] < 60) & (gdf["minor_length_m"] > 1)
]

In [120]:
len(gdf_filter)

55

## Visualiza final results

In [121]:
geoai.view_vector_interactive(gdf_filter, column="confidence", tiles=raster_url)

In [122]:
geoai.view_vector_interactive(gdf_filter, tiles=raster_url)

![image](https://github.com/user-attachments/assets/a1e4c871-b152-466a-b902-7c00e62e5444)